# Prompt Engineering con OpenAI

En este notebook veremos como utilizar la API de OpenAI para aplicar diferentes técnicas de prompt engineering:

In [ ]:
# 👉 Ejecuta esta celda una sola vez para instalar dependencias
!pip install -q openai ipywidgets python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.4 MB/s eta 0:00:00


Accedemos a la web de la [API de OpenAI](https://platform.openai.com/docs/overview) para obtener una API KEY que podamos utlizar

In [ ]:
# 👉 Introduce tu clave de OpenAI (se guarda solo en la sesión actual)
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("🔑 OpenAI API key: ")
print("✅ Clave configurada.")


🔑 OpenAI API key: ··········
✅ Clave configurada.


## 1. Introducción al Prompt Engineering  
El **prompt engineering** es la práctica de diseñar instrucciones (prompts) para maximizar la **utilidad, precisión y seguridad** de las respuestas de los modelos de lenguaje de gran escala (LLMs).

Problemas que aborda:
- **Alucinaciones**: las respuestas inventadas o incorrectas.  
- **Ambigüedad**: enunciados poco claros generan salidas inconsistentes.  
- **Dependencia de contexto**: la falta de información relevante degrada la calidad.  

A continuación probaremos un ejemplo simple.


In [ ]:
from openai import OpenAI
import os

client = OpenAI()

def ask(prompt, model="gpt-4.1-nano", max_tokens=150):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role":"user","content":prompt}],
        seed=42
        #temperature=0.7,
        #max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()


In [ ]:
ejemplo = "¿Qué es el prompt engineering en una frase corta y clara?"
print(ask(ejemplo))

## 2. Instruction‑Based Prompting  
Como hemos visto en la teoría, un **prompt** bien estructurado puede incluir varios bloques:  

| Bloque | Descripción | Ejemplo |
| --- | --- | --- |
| **Persona** | Rol o estilo de la IA | «Eres un profesor paciente…» |
| **Instrucción** | Tarea a realizar | «Explica la fotosíntesis…» |
| **Contexto** | Información adicional | «Para estudiantes de 12 años…» |
| **Formato** | Forma de la salida | «Responde en bullets…» |
| **Audiencia** | A quién va dirigido | «Dirígete a tus alumnos…» |
| **Tono** | Voz deseada | «Usa tono informal…» |
| **Datos** | Ejemplos, tablas, etc. | (opcional) |

La construcción del prompt es **iterativa**. Veamos un ejemplo paso a paso.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

persona     = widgets.Text(value='Eres un experto en IA experto en comunicación adaptada al nivel de tu audiencia.', description='Persona')
instruccion = widgets.Textarea(value='Explica la Self-Attention de la arquitectura Transformers', description='Instrucción')
contexto    = widgets.Textarea(value='', description='Contexto')
formato     = widgets.Text(value='Explica el proceso en pasos numerados.', description='Formato')
audiencia   = widgets.Text(value='Estudiantes de primaria', description='Audiencia')
tono        = widgets.Text(value='Tono gracioso.', description='Tono')
btn = widgets.Button(description="Generar respuesta")

out = widgets.Output()

def build_prompt(_):
    prompt = "\n".join([persona.value,
                         instruccion.value,
                         contexto.value,
                         formato.value,
                         audiencia.value,
                         tono.value]).strip()
    with out:
        out.clear_output()
        display(Markdown(f"**Prompt generado:**\n```\n{prompt}\n```"))
        respuesta = ask(prompt)
        display(Markdown(f"**Respuesta del modelo:**\n{respuesta}"))

btn.on_click(build_prompt)

display(widgets.VBox([persona, instruccion, contexto, formato, audiencia, tono, btn, out]))


## 3. In‑Context Learning (ICL)  
En lugar de entrenar el modelo de nuevo, se **enseña con ejemplos en el mismo prompt**:

- **Zero‑shot**: sin ejemplos.  
- **One‑shot**: un ejemplo.  
- **Few‑shot**: varios ejemplos.

El modelo aprende del patrón observado para **generalizar** a la nueva consulta.


In [ ]:
import json

ej_bio = [
   'Concepto: "CPU" → Analogía: "Es como el cerebro que decide qué hacer en una fábrica."',
   'Concepto: "Red neuronal" → Analogía: "Es como un grupo de trabajadores que votan cuál es la mejor solución."'
]

texto_desafio = (
    "servidor web",
    "GPU"
)

# Zero-shot (sin ejemplos)
ej_zero = []

# One-shot (un ejemplo sencillo)
ej_one = [ej_bio[0]]

# Few-shot (varios ejemplos, incluido uno con alias y evento)
ej_few = ej_bio

concepto = texto_desafio[0]
prompt = f"Crear una analogía para explicar el concepto{concepto}.\n"
ejemplos_one = f"Aquí tienes un ejemplo:\n {ej_one}"
ejemplos_few = f"Aquí tienes algunos ejemplos:\n {ej_few}"

print("Zero-shot: \n", ask(prompt))
print("One-shot: \n", ask(prompt+ejemplos_one))
print("Few-shot: \n", ask(prompt+ejemplos_few))


## 4. Técnicas Avanzadas  

### 4.1 Chain‑of‑Thought (CoT)  
Se le pide al modelo que **razone paso a paso** antes de entregar la respuesta final.


In [ ]:
# Demostración CoT
prompt = '''
Eres un experto matemático.
Si tengo 3 manzanas y compro 5 más, y luego regalo 4, ¿cuántas me quedan?

'''

cot_prompt = '''
Eres un experto matemático.
Si tengo 3 manzanas y compro 5 más, y luego regalo 4, ¿cuántas me quedan?

Think step by step
'''



Incluso sin utilizar el "think step by step", el modelo ya ha sido entrenado para detectar este tipo de situación y aplicar reasoning durante el entrenamiento.

In [ ]:
print(ask(prompt))


Comenzaste con 3 manzanas. Luego compraste 5 más, así que:

3 + 5 = 8 manzanas.

Luego regalaste 4, por lo que te quedan:

8 - 4 = 4 manzanas.

Respuesta: Te quedan **4 manzanas**.


In [ ]:
print(ask(cot_prompt))


Vamos a resolverlo paso a paso:

1. Empiezas con 3 manzanas.
2. Compras 5 más, por lo tanto, tienes: 3 + 5 = 8 manzanas.
3. Luego regalas 4, así que te quedan: 8 - 4 = 4 manzanas.

Respuesta: **Te quedan 4 manzanas**.


### Podemos proporcioar al prompt una guía de como llevar a cabo la tarea (Chain of Thought Guiado)

In [ ]:
frase = "El anticuerpo monoclonal Rituximab, utilizado en el tratamiento del linfoma no Hodgkin, actúa uniéndose al antígeno CD20 presente en la superficie de células B malignas, inhibiendo su proliferación."

In [ ]:
prompt = f'''
Dado el siguiente texto biomédico:

{frase}

Identifica primero las entidades relevantes y luego las relaciones entre ellas.
Sigue estos pasos de razonamiento:

1. **Detecta todas las entidades biomédicas** mencionadas en el texto (por ejemplo, enfermedades, genes, proteínas, medicamentos, procedimientos).
2. **Clasifica** cada entidad con su tipo (por ejemplo, "ENFERMEDAD", "GEN", "MEDICAMENTO", "PROTEINA", "PROCEDIMIENTO").
3. **Identifica las relaciones** que se mencionan entre las entidades (por ejemplo, "trata", "causa", "inhibe", "asocia con", "codifica", etc.).
4. Escribe el resultado en dos partes:
   - Una lista de entidades (cada una con su texto y tipo).
   - Una lista de relaciones (cada una indicando entidad origen, relación y entidad destino).

Devuelve una respuesta en formato JSON como este:

```json
{{
    "entidades": [
        {{
            "texto": "nombre de la entidad",
            "tipo": "tipo de entidad",
            "comentarios": "comentarios sobre la eleccion"
        }}
    ],
    "relaciones": [
        {{
            "origen": "texto de entidad origen",
            "relacion": "tipo de relación",
            "destino": "texto de entidad destino"
        }}
    ]
}}
```

Ejemplo:
"El fármaco Imatinib inhibe la actividad de la proteína BCR-ABL, asociada a la leucemia mieloide crónica."

**Paso a paso:**

**1. Detectar entidades:**
- Imatinib
- BCR-ABL
- leucemia mieloide crónica

**2. Clasificar entidades:**
- Imatinib → MEDICAMENTO
- BCR-ABL → PROTEINA
- leucemia mieloide crónica → ENFERMEDAD

**3. Identificar relaciones:**
- Imatinib inhibe BCR-ABL
- BCR-ABL se asocia con leucemia mieloide crónica

**4. Resultado final en JSON:**

```json
{{
    "entidades": [
        {{ "texto": "Imatinib", "tipo": "MEDICAMENTO", "comentarios": "Seha elegido porque.." }},
        {{ "texto": "BCR-ABL", "tipo": "PROTEINA", "comentarios": "Ejemplo de porque se ha elegido" }},
        {{ "texto": "leucemia mieloide crónica", "tipo": "ENFERMEDAD", "comentarios": "Ejemplo de porque se ha elegido" }}
    ],
    "relaciones": [
        {{ "origen": "Imatinib", "relacion": "inhibe", "destino": "BCR-ABL" }},
        {{ "origen": "BCR-ABL", "relacion": "asociada a", "destino": "leucemia mieloide crónica" }}
    ]
}}
```
'''

In [ ]:
result = ask(prompt)

In [ ]:
import json
json.loads(result)

{'entidades': [{'texto': 'Rituximab',
   'tipo': 'MEDICAMENTO',
   'comentarios': 'Es un anticuerpo monoclonal utilizado en el tratamiento del linfoma no Hodgkin'},
  {'texto': 'linfoma no Hodgkin',
   'tipo': 'ENFERMEDAD',
   'comentarios': 'Es la enfermedad que trata Rituximab'},
  {'texto': 'antígeno CD20',
   'tipo': 'PROTEINA',
   'comentarios': 'Es la proteína específica a la que se une Rituximab'},
  {'texto': 'células B malignas',
   'tipo': 'CELULA',
   'comentarios': 'Tipo de célula que expresa el antígeno CD20 y que es afectada en la enfermedad'}],
 'relaciones': [{'origen': 'Rituximab',
   'relacion': 'se une a',
   'destino': 'antígeno CD20'},
  {'origen': 'Rituximab',
   'relacion': 'actúa en',
   'destino': 'linfoma no Hodgkin'},
  {'origen': 'antígeno CD20',
   'relacion': 'está presente en',
   'destino': 'células B malignas'},
  {'origen': 'Rituximab',
   'relacion': 'inhibe la proliferación de',
   'destino': 'células B malignas'}]}

## 5. Usos de Prompt Engineering  

- **Pre‑anotación de datos**: acelerar el etiquetado en NLP.  


Sin formato, no lo entiende bien:

In [ ]:
texto = "Uber planea lanzar coches autónomos en Madrid el próximo año, afirma su CEO Dara Khosrowshahi."
etiquetado_prompt = f'''
Etiqueta las entidades PERSONA, ORGANIZACION y LOCALIZACION en el texto.
Texto: {texto}
'''
print(ask(etiquetado_prompt))


Uber planea lanzar coches autónomos en LOCALIZACION Madrid el próximo año, afirma su PERSONA su CEO Dara Khosrowshahi.


In [ ]:
texto = "Uber planea lanzar coches autónomos en Madrid el próximo año, afirma su CEO Dara Khosrowshahi."
etiquetado_prompt = f'''
Etiqueta las entidades PERSONA, ORGANIZACION y LOCALIZACION en el texto.
Devuelve un JSON con las entidades y su categoría.
Texto: {texto}
'''
print(ask(etiquetado_prompt))



{
  "entidades": [
    {
      "nombre": "Uber",
      "categoría": "ORGANIZACION"
    },
    {
      "nombre": "Madrid",
      "categoría": "LOCALIZACION"
    },
    {
      "nombre": "Dara Khosrowshahi",
      "categoría": "PERSONA"
    }
  ]
}


Podeos definir json específico para asegurar formato:

In [ ]:
texto = "Uber planea lanzar coches autónomos en Madrid el próximo año, afirma su CEO Dara Khosrowshahi."
etiquetado_prompt = f'''
Etiqueta las entidades PERSONA, ORGANIZACION y LOCALIZACION en el texto.
Devuelve una estructura json como la siguiente. No añadas más información además del json:
```json
[
    {{
        "texto": "nombre de la entidad extraída",
        "tipo": "categoría asignada"
    }}
]
```

Texto: {texto}
'''
resultado = ask(etiquetado_prompt)


In [ ]:
print(resultado)


[
    {
        "texto": "Uber",
        "tipo": "ORGANIZACION"
    },
    {
        "texto": "Madrid",
        "tipo": "LOCALIZACION"
    },
    {
        "texto": "Dara Khosrowshahi",
        "tipo": "PERSONA"
    }
]


In [ ]:
eval(resultado)[0]["texto"]

'Uber'

- **Generación de datos sintéticos**: crear ejemplos adicionales para mejorar modelos.  


In [ ]:
sint_prompt = '''
Genera 3 oraciones adicionales que expresen satisfacción del cliente con un servicio de entrega de comida.
'''
print(ask(sint_prompt))


1. ¡Estoy muy satisfecho con la rapidez y la calidad de la entrega, definitivamente volveré a ordenar aquí!  
2. La comida llegó caliente y en perfectas condiciones, excelente servicio.  
3. El repartidor fue muy amable y la entrega fue más rápida de lo esperado, ¡gracias por la excelente atención!
